In [8]:
%pip install PyPDF2  # for reading PDFs
import PyPDF2

# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
        return text

# Path to your PDF file
pdf_path = "/content/Sample Data for Evaluation.pdf"
document_text = extract_text_from_pdf(pdf_path)
print(document_text)

Unacceptable Risk  
Scenario: Social Scoring System  
• Description: A government implements an AI system that assigns scores to citizens based on 
their social behavior, ﬁnancial status, and online activities.  
• Risk: This could lead to discrimination, loss of privacy, and social exclusion, making it an 
unacceptable risk under the EU AI Act.  
High Risk  
Scenario: AI in Healthcare  
• Description: A hospital uses an AI system to diagnose diseases and recommend treatments 
based on patient data.  
• Risk: Incorrect diagnoses or treatment recommendations could have serious health 
consequences. Therefore, this AI system is subject to strict regulations and oversight.  
Scenario: Autonomous Vehicles  
• Description: A company deploys self -driving cars on public roads.  
• Risk: Malfunctions or errors in the AI system could lead to accidents, posing signiﬁcant safety 
risks. These systems must comply with rigorous safety standards.  
Limited Risk  
Scenario: Customer Service Chatbots

In [14]:
%pip install llama-index-llms-openai
!pip install llama-index
!pip install pymilvus
import os
import openai
import textwrap
from llama_index.llms.openai import OpenAI
from llama_index.core import GPTVectorStoreIndex, SimpleDirectoryReader, Document
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection

openai.api_key = "4b3ea2d5074c4059ad670e016e83b853"


In [15]:
# # creating RAG database and tagging data with metadata

# documents = [Document(text = document_text)]

# # initialize LlamaIndex and creates an index
# index = GPTVectorStoreIndex.from_documents(documents)

# # generates metadata tags from index
# # you would do this with all documents, but for now there's just one
# tags = index.get_keywords()
# print(tags)

# Connect to Milvus
ENDPOINT = "https://in03-d592609349d65df.serverless.gcp-us-west1.cloud.zilliz.com"
TOKEN = "c96536849f835f69648e8e1586f3e09794e9d9c63c2375d55bb62aa5a7b031bba566b53d6afd4410bb85f4aeea14f1406b947fe5"
connections.connect(
   uri = ENDPOINT,
   token = TOKEN)

# Define schema for Milvus collection
fields = [
    FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True),
    FieldSchema(name = "embedding", dtype = DataType.FLOAT_VECTOR, dim = 1536),
    FieldSchema(name = "text", dtype = DataType.VARCHAR, max_length = 65535)
]

schema = CollectionSchema(fields, "EU AI Act Collection")
collection = Collection("eu_ai_act", schema)

# Split the text into smaller chunks (e.g., 5000 characters)
chunk_size = 5000
chunks = textwrap.wrap(document_text, chunk_size)

# Create individual documents from chunks
documents = [Document(text=chunk) for chunk in chunks]

# Build the vector index using LlamaIndex (GPTVectorStoreIndex)
index = GPTVectorStoreIndex(documents)

# Store document embeddings in Milvus
embeddings = index.get_vector_store().get_embeddings()  # Extract embeddings
texts = [node.text for node in index.get_vector_store().nodes]  # Corresponding text chunks

# Prepare data for insertion into Milvus
data = [
    list(range(len(embeddings))),  # IDs
    embeddings,                    # Embeddings
    texts                          # Text chunks
]

# Insert into Milvus
collection.insert(data)
collection.load()

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
import numpy as np
# Perform a similarity search based on a user query
# Define a query function to retrieve relevant text chunks based on the user query

def query_milvus(user_query: str):
    # Generate the query embedding using OpenAI's embeddings API
    query_embedding = openai.Embedding.create(
        input=user_query,
        model="text-embedding-ada-002"
    )['data'][0]['embedding']

    # Convert query embedding to numpy array
    query_embedding = np.array(query_embedding).reshape(1, -1)

    # Perform vector similarity search in Milvus
    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
    results = collection.search(
        data=query_embedding,
        anns_field="embedding",
        param=search_params,
        limit=3,
        output_fields=["text"]
    )

    # Extract and return the most relevant text chunks
    return [hit.entity.get("text") for hit in results[0]]

# Example query
user_query = "What are the risks associated with AI?"
relevant_text_chunks = query_milvus(user_query)

# Print the top retrieved text chunks from Milvus
for chunk in relevant_text_chunks:
    print(chunk)
